In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping


2024-09-17 18:33:10.974183: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-17 18:33:11.184907: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 18:33:11.275370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 18:33:11.302088: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 18:33:11.462742: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Define image size and batch size
IMG_SIZE = (224, 224)  # Image size for VGG16
BATCH_SIZE = 32

# Paths to your dataset
train_dir = '/home/esidserver/datasets/testing_cls/training-v2/train/'
val_dir = '/home/esidserver/datasets/testing_cls/training-v2/val/'
test_dir = '/home/esidserver/datasets/testing_cls/training-v2/test/'

In [3]:
# Data augmentation and rescaling
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0/255)

# Load images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=['AB', 'AG', 'ANO', 'CQ']  # Specify the class names
)


Found 5193 images belonging to 4 classes.
Found 1484 images belonging to 4 classes.
Found 740 images belonging to 4 classes.


In [4]:
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [5]:
# Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers (don't train them initially)
base_model.trainable = False

# Create a new model on top of it
model = models.Sequential([
    base_model,
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(4, activation='softmax')  # 4 output classes
])

# Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Set up a checkpoint to save the best model
checkpoint = ModelCheckpoint('best_vgg16_model.keras', monitor='val_accuracy', save_best_only=True, mode='max')

#Set up a earlystop to save the best model
early_stopping=EarlyStopping(monitor='val_accuracy', patience=50, restore_best_weights=True)

# Set up a checkpoint to save the best model
#checkpoint = ModelCheckpoint('best_vgg16_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')

2024-09-17 18:33:15.970891: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22453 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:af:00.0, compute capability: 8.6


In [6]:
#pip install --upgrade tensorflow tensorflow-gpu
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))



Num GPUs Available:  1


In [7]:
#auto early stop if the efficiecy is not improving for 50 epochs
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=250,  # Adjust the number of epochs based on your needs
    callbacks=[checkpoint]
)

# Unfreeze the base model for fine-tuning
base_model.trainable = True




Epoch 1/250


/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1726572804.844468   52743 service.cc:146] XLA service 0x73576400d950 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726572804.844513   52743 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3090, Compute Capability 8.6
2024-09-17 18:33:24.912312: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-17 18:33:25.335059: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8902


  1/162 ━━━━━━━━━━━━━━━━━━━━ 28:58 11s/step - accuracy: 0.2812 - loss: 1.9598

I0000 00:00:1726572810.021042   52743 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


162/162 ━━━━━━━━━━━━━━━━━━━━ 92s 502ms/step - accuracy: 0.4800 - loss: 1.2744 - val_accuracy: 0.7439 - val_loss: 0.6398
Epoch 2/250
  1/162 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - accuracy: 0.6875 - loss: 0.8604

...

2024-09-17 20:40:55.957630: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 20:40:55.987610: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 20:40:55.987713: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


162/162 ━━━━━━━━━━━━━━━━━━━━ 60s 354ms/step - accuracy: 0.9398 - loss: 0.1673 - val_accuracy: 0.9572 - val_loss: 0.1244
Epoch 248/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 284us/step - accuracy: 1.0000 - loss: 0.0808 - val_accuracy: 1.0000 - val_loss: 0.0604
Epoch 249/250


2024-09-17 20:41:55.935886: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 20:41:55.935956: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882
2024-09-17 20:41:55.965749: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 20:41:55.965854: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


162/162 ━━━━━━━━━━━━━━━━━━━━ 62s 364ms/step - accuracy: 0.9337 - loss: 0.1684 - val_accuracy: 0.9613 - val_loss: 0.1185
Epoch 250/250
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.9375 - loss: 0.1434 - val_accuracy: 1.0000 - val_loss: 0.0164


2024-09-17 20:42:57.631590: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 20:42:57.631711: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


In [8]:
# Recompile the model with a lower learning rate for fine-tuning
model.compile(
    optimizer=optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Fine-tune the model
fine_tune_history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=100,  # Further epochs for fine-tuning
    callbacks=[checkpoint]
)


Epoch 1/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 90s 405ms/step - accuracy: 0.9178 - loss: 0.2378 - val_accuracy: 0.9402 - val_loss: 0.2008
Epoch 2/100
  1/162 ━━━━━━━━━━━━━━━━━━━━ 18s 113ms/step - accuracy: 0.9062 - loss: 0.2339

2024-09-17 20:44:28.397271: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505


162/162 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.9062 - loss: 0.2339 - val_accuracy: 0.8333 - val_loss: 0.6143
Epoch 3/100


2024-09-17 20:44:29.810927: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 20:44:29.811047: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 346ms/step - accuracy: 0.9512 - loss: 0.1233 - val_accuracy: 0.9545 - val_loss: 0.1342
Epoch 4/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 219us/step - accuracy: 0.8438 - loss: 0.5743 - val_accuracy: 1.0000 - val_loss: 0.0042
Epoch 5/100


...

2024-09-17 21:31:11.490995: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 21:31:11.515262: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 21:31:11.515360: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


162/162 ━━━━━━━━━━━━━━━━━━━━ 59s 346ms/step - accuracy: 0.9917 - loss: 0.0209 - val_accuracy: 0.9851 - val_loss: 0.0771
Epoch 98/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 188us/step - accuracy: 1.0000 - loss: 0.0011 - val_accuracy: 0.9167 - val_loss: 0.0999
Epoch 99/100


2024-09-17 21:32:10.353653: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 21:32:10.375397: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 21:32:10.375495: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


162/162 ━━━━━━━━━━━━━━━━━━━━ 60s 355ms/step - accuracy: 0.9942 - loss: 0.0172 - val_accuracy: 0.9803 - val_loss: 0.1640
Epoch 100/100
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 272us/step - accuracy: 1.0000 - loss: 0.0010 - val_accuracy: 1.0000 - val_loss: 7.6491e-06


2024-09-17 21:33:10.632807: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 21:33:10.632923: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882
2024-09-17 21:33:10.658721: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 2351489927790708505
2024-09-17 21:33:10.658824: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 13646629611367104882


In [9]:
# Load the best saved model
model.load_weights('best_vgg16_model.keras')

# Evaluate the model on the validation set
val_loss, val_accuracy = model.evaluate(val_generator)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Accuracy: {test_accuracy:.4f}")

/home/esidserver/miniforge3/envs/tf/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'adam', because it has 62 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


47/47 ━━━━━━━━━━━━━━━━━━━━ 4s 92ms/step - accuracy: 0.8882 - loss: 0.3167
Validation Accuracy: 0.8814
24/24 ━━━━━━━━━━━━━━━━━━━━ 5s 193ms/step - accuracy: 0.7762 - loss: 0.5517
Test Accuracy: 0.7878
